In [ ]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go

from cc_hardware.drivers.spads import SPADDataType, SPADSensorData
from cc_hardware.drivers.spads.pkl import PklSPADSensorConfig, PklSPADSensor

In [ ]:
pkl_path = Path("../../../logs/20240922_192338/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/10-38-07/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/11-52-34/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/11-54-54/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/13-49-54/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/13-58-09/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/13-59-08/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/18-16-03/data.pkl")
pkl_path = Path("../../../logs/2025-05-25/18-19-19/data.pkl")
assert pkl_path.exists()

sensor = PklSPADSensor(PklSPADSensorConfig.create(pkl_path=pkl_path, loop=True, width=8, height=8, num_bins=18, fovx=45, fovy=45, timing_resolution=250e-12, data_type=SPADDataType.HISTOGRAM | SPADDataType.POINT_CLOUD | SPADDataType.DISTANCE))
height, width = sensor.resolution
print(f"Resolution: {height} x {width}")

index = 10

In [ ]:
histograms = sensor.accumulate(index=index)[SPADDataType.HISTOGRAM]

ylim = None # 1e4
min_bin = 0
max_bin = min(histograms.shape[-1] - 1, 60)

def plot_histogram(i: int, j: int):
    plt.subplot(height, width, i * height + j + 1)
    plt.plot(histograms[i, j])
    # plt.title(f"i={i}, j={j}")
    # plt.yscale("log")

    plt.xlabel("")
    plt.xticks([])
    plt.yticks([])
    plt.ylabel("")

    if ylim is not None:
        plt.ylim(None, ylim)
    if min_bin is not None:
        plt.xlim(min_bin, None)
    if max_bin is not None:
        plt.xlim(None, max_bin)

plt.figure(figsize=(height * 2, width * 2), dpi=400)
for i in range(height):
    for j in range(width):
        plot_histogram(i, j)
plt.tight_layout()

In [ ]:
sensor_data = SPADSensorData(sensor.config)
sensor_data.process(sensor.accumulate(index=index))
data = sensor_data.get_data()
distances = data[SPADDataType.DISTANCE]
pt_cloud = sensor_data.calculate_point_cloud(distances=distances, subpixel_samples=5, bilinear_interpolation=True)

# Flatten the point cloud for Plotly
x = pt_cloud[..., 0].flatten()
y = pt_cloud[..., 1].flatten()
z = pt_cloud[..., 2].flatten()

fig = go.Figure(data=
    [
        go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(color='blue', size=3)),
    ]
)

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'  # or 'cube' or 'manual' with 'aspectratio'
    )
)

fig.show()

In [ ]:
all_points = []

for i in range(len(sensor.handler)):
    sensor_data = SPADSensorData(sensor.config)
    sensor_data.process(sensor.accumulate(index=i))
    data = sensor_data.get_data()
    distances = data[SPADDataType.DISTANCE]
    pt_cloud = sensor_data.calculate_point_cloud(distances=distances, subpixel_samples=5, bilinear_interpolation=True)
    all_points.append(pt_cloud.reshape(-1, 3))

combined_points = np.concatenate(all_points, axis=0)
x, y, z = combined_points[:, 0], combined_points[:, 1], combined_points[:, 2]

fig = go.Figure(data=[
    go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(color='blue', size=3))
])

fig.update_layout(
    scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z', aspectmode='data')
)

fig.show()
